In [ ]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 1000

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def greedy(self, state):
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        states, targets_f = [], []
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target 
            # Filtering out states and targets for training
            states.append(state[0])
            targets_f.append(target_f[0])
        #print(np.array(states).shape,np.array(targets_f).shape)
        #print(np.array(states), np.array(targets_f))
        #print("-----")
        history = self.model.fit(np.array(states), np.array(targets_f), epochs=1, verbose=0)
        # Keeping track of loss
        loss = history.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)



env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            loss = agent.replay(batch_size)
            # Logging training loss every 10 timesteps
            if time % 10 == 0:
                print("episode: {}/{}, time: {}, loss: {:.4f}"
                    .format(e, EPISODES, time, loss))  
    #if e % 10 == 0:
    #    agent.save("./save/cartpole-dqn.h5")


episode: 0/1000, score: 19, e: 1.0
Instructions for updating:
Use tf.cast instead.
episode: 1/1000, score: 15, e: 0.99
episode: 2/1000, time: 0, loss: 2.0744
episode: 2/1000, score: 9, e: 0.94
episode: 3/1000, time: 0, loss: 5.2632
episode: 3/1000, time: 10, loss: 3.8246
episode: 3/1000, time: 20, loss: 3.7994
episode: 3/1000, time: 30, loss: 2.0912
episode: 3/1000, time: 40, loss: 0.6529
episode: 3/1000, time: 50, loss: 5.2653
episode: 3/1000, time: 60, loss: 0.8588
episode: 3/1000, time: 70, loss: 2.3081
episode: 3/1000, time: 80, loss: 1.3363
episode: 3/1000, score: 83, e: 0.62
episode: 4/1000, time: 0, loss: 1.3124
episode: 4/1000, time: 10, loss: 1.2931
episode: 4/1000, score: 14, e: 0.58
episode: 5/1000, time: 0, loss: 4.7287
episode: 5/1000, time: 10, loss: 6.2376
episode: 5/1000, score: 16, e: 0.53
episode: 6/1000, time: 0, loss: 1.5032
episode: 6/1000, time: 10, loss: 6.0523
episode: 6/1000, score: 13, e: 0.5
episode: 7/1000, time: 0, loss: 3.2887
episode: 7/1000, score: 9, e:

episode: 37/1000, time: 20, loss: 0.2677
episode: 37/1000, time: 30, loss: 0.4557
episode: 37/1000, time: 40, loss: 0.5476
episode: 37/1000, time: 50, loss: 16.0560
episode: 37/1000, score: 57, e: 0.01
episode: 38/1000, time: 0, loss: 2.8129
episode: 38/1000, time: 10, loss: 1.3514
episode: 38/1000, time: 20, loss: 7.7728
episode: 38/1000, time: 30, loss: 3.9265
episode: 38/1000, time: 40, loss: 0.2482
episode: 38/1000, time: 50, loss: 4.0073
episode: 38/1000, time: 60, loss: 1.0985
episode: 38/1000, time: 70, loss: 6.9647
episode: 38/1000, time: 80, loss: 0.1956
episode: 38/1000, time: 90, loss: 1.4005
episode: 38/1000, time: 100, loss: 0.2188
episode: 38/1000, time: 110, loss: 22.3714
episode: 38/1000, score: 114, e: 0.01
episode: 39/1000, time: 0, loss: 0.1761
episode: 39/1000, time: 10, loss: 0.6662
episode: 39/1000, time: 20, loss: 2.9554
episode: 39/1000, time: 30, loss: 0.2486
episode: 39/1000, time: 40, loss: 12.6307
episode: 39/1000, time: 50, loss: 0.1043
episode: 39/1000, ti

In [ ]:
# Play game
print("\nPlaying Game...")
sleep(1)

s = env.reset()
done = False
while not done:
    env.render()
    a = agent.greedy(np.array([s]))
    newS, r, done, _ = env.step(a)
    s = newS